In [1]:
!unzip captcha-recognition-pytorch-master.zip

Archive:  captcha-recognition-pytorch-master.zip
   creating: captcha-recognition-pytorch-master/
  inflating: captcha-recognition-pytorch-master/config.py  
  inflating: captcha-recognition-pytorch-master/dataset.py  
  inflating: captcha-recognition-pytorch-master/engine.py  
   creating: captcha-recognition-pytorch-master/input/
 extracting: captcha-recognition-pytorch-master/input/.keep  
  inflating: captcha-recognition-pytorch-master/model.py  
  inflating: captcha-recognition-pytorch-master/train.py  


In [2]:
cd /content/

/content


In [3]:
!mkdir ~/.kaggle

In [4]:
cd ~/.kaggle/

/root/.kaggle


In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"geghamshaqaryan","key":"fc29f9ea91bc6eed59a7e7f57e286d74"}'}

In [6]:
!chmod 600 /root/.kaggle/kaggle.json

In [7]:
cd /content/captcha-recognition-pytorch-master/input/

/content/captcha-recognition-pytorch-master/input


In [8]:
!kaggle datasets download 'parsasam/captcha-dataset'

 97% 345M/356M [00:16<00:00, 17.3MB/s]
100% 356M/356M [00:16<00:00, 22.8MB/s]


In [9]:
!unzip captcha-dataset.zip -d /content/captcha-recognition-pytorch-master/input/

Streaming output truncated to the last 5000 lines.
  inflating: /content/captcha-recognition-pytorch-master/input/xK1mM.jpg  
  inflating: /content/captcha-recognition-pytorch-master/input/xK2Q6.jpg  
  inflating: /content/captcha-recognition-pytorch-master/input/xK3Yl.jpg  
  inflating: /content/captcha-recognition-pytorch-master/input/xK5fU.jpg  
  inflating: /content/captcha-recognition-pytorch-master/input/xKA6K.jpg  
  inflating: /content/captcha-recognition-pytorch-master/input/xKEYy.jpg  
  inflating: /content/captcha-recognition-pytorch-master/input/xKF1u.jpg  
  inflating: /content/captcha-recognition-pytorch-master/input/xKJ3w.jpg  
  inflating: /content/captcha-recognition-pytorch-master/input/xKMhX.jpg  
  inflating: /content/captcha-recognition-pytorch-master/input/xKO5H.jpg  
  inflating: /content/captcha-recognition-pytorch-master/input/xKRRk.jpg  
  inflating: /content/captcha-recognition-pytorch-master/input/xKRzV.jpg  
  inflating: /content/captcha-recognition-pytorch

In [10]:
cd /content/captcha-recognition-pytorch-master/

/content/captcha-recognition-pytorch-master


In [11]:
!python model.py

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Traceback (most recent call last):
  File "model.py", line 51, in <module>
    x, _ = cm(img, torch.rand((1, 5)))
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "model.py", line 26, in forward
    x = F.relu(self.linear_1(x))
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/linear.py", line 96, in forward
    return F.linear(input, s

In [21]:
import torch
torch.cuda.is_available()

True

In [49]:
!python config.py

In [18]:
!python train.py

  0% 0/12720 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100% 12720/12720 [19:26<00:00, 10.90it/s]
100% 1414/1414 [00:42<00:00, 33.29it/s]
[('7ZNb7', 'Mw'), ('OULk3', '1F'), ('PktWE', 'WwW'), ('Zmxjh', 'MwWw'), ('bpzSD', '1F'), ('Aeaeu', 'g'), ('C32cU', '1ji'), ('lkIgN', 'g'), ('a7WR5', 'MmAm'), ('ErzCl', '1ji')]
Epoch=0, Train Loss=4.357738586754169, Test Loss=4.143820486446418 Accuracy=0.0
100% 12720/12720 [19:24<00:00, 10.92it/s]
100% 1414/1414 [00:42<00:00, 33.47it/s]
[('7ZNb7', 'Ky'), ('OULk3', 'E2S'), ('PktWE', 'NWM'), ('Zmxjh', 'W8mu'), ('bpzSD', 'ezZ2S'), ('Aeaeu', 'MAm'), ('C32cU', '1z7zf'), ('lkIgN', 'HDKW'

In [50]:
import os
import glob
import torch
import numpy as np

import albumentations
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics

import config
import dataset
import engine
from model import CaptchaModel


from torch import nn

In [51]:
def remove_duplicates(x):
    if len(x) < 2:
        return x
    fin = ""
    for j in x:
        if fin == "":
            fin = j
        else:
            if j == fin[-1]:
                continue
            else:
                fin = fin + j
    return fin


def decode_predictions(preds, encoder):
    preds = preds.permute(1, 0, 2)
    preds = torch.softmax(preds, 2)
    preds = torch.argmax(preds, 2)
    preds = preds.detach().cpu().numpy()
    cap_preds = []
    for j in range(preds.shape[0]):
        temp = []
        for k in preds[j, :]:
            k = k - 1
            if k == -1:
                temp.append("§")
            else:
                p = encoder.inverse_transform([k])[0]
                temp.append(p)
        tp = "".join(temp).replace("§", "")
        cap_preds.append(remove_duplicates(tp))
    return cap_preds

In [52]:
image_files = glob.glob(os.path.join(config.DATA_DIR, "*.jpg"))
targets_orig = [x.split("/")[-1][:-4] for x in image_files]
targets = [[c for c in x] for x in targets_orig]
targets_flat = [c for clist in targets for c in clist]

lbl_enc = preprocessing.LabelEncoder()
lbl_enc.fit(targets_flat)
targets_enc = [lbl_enc.transform(x) for x in targets]
targets_enc = np.array(targets_enc)
targets_enc = targets_enc + 1
(
    train_imgs,
    test_imgs,
    train_targets,
    test_targets,
    _,
    test_targets_orig,
) = model_selection.train_test_split(
    image_files, targets_enc, targets_orig, test_size=0.1, random_state=42
)

train_dataset = dataset.ClassificationDataset(
    image_paths=train_imgs,
    targets=train_targets,
    resize=(config.IMAGE_HEIGHT, config.IMAGE_WIDTH),
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    shuffle=True,
)
test_dataset = dataset.ClassificationDataset(
    image_paths=test_imgs,
    targets=test_targets,
    resize=(config.IMAGE_HEIGHT, config.IMAGE_WIDTH),
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    shuffle=False,
)

#MODEL EVAL

In [53]:
model = CaptchaModel(num_chars=len(lbl_enc.classes_))
model.to(config.DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.8, patience=5, verbose=True
)

#MOUNT DRIVE

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
drive_dir = '/content/drive/MyDrive/Colab Notebooks/'

#LOAD MODEL PARAMS

In [55]:
state = torch.load(f'{drive_dir}state_10.pth')
model.load_state_dict(state['state_dict'])
optimizer.load_state_dict(state['optimizer'])

In [57]:
config.EPOCHS = 10

In [58]:
config.EPOCHS

10

#TRAIN AND SAVE MODEL PARAMS 

In [ ]:
for epoch in range(config.EPOCHS):
    train_loss = engine.train_fn(model, train_loader, optimizer)
    valid_preds, test_loss = engine.eval_fn(model, test_loader)
    valid_captcha_preds = []
    for vp in valid_preds:
        current_preds = decode_predictions(vp, lbl_enc)
        valid_captcha_preds.extend(current_preds)
    combined = list(zip(test_targets_orig, valid_captcha_preds))
    print(combined[:10])
    test_dup_rem = [remove_duplicates(c) for c in test_targets_orig]
    accuracy = metrics.accuracy_score(test_dup_rem, valid_captcha_preds)
    print(
        f"Epoch={epoch}, Train Loss={train_loss}, Test Loss={test_loss} Accuracy={accuracy}"
    )
    scheduler.step(test_loss)

100%|██████████| 1414/1414 [00:45<00:00, 31.30it/s]


[('7ZNb7', '7zNAZ'), ('OULk3', 'Gnmx7'), ('PktWE', 'MmWMR'), ('Zmxjh', '4HAu'), ('bpzSD', 'RPZGA'), ('Aeaeu', 'AGan'), ('C32cU', 'Cez4H'), ('lkIgN', 'TjNOV'), ('a7WR5', 'cwnr2'), ('ErzCl', 'kHcGx')]
Epoch=0, Train Loss=3.134764814095677, Test Loss=3.4087534126163033 Accuracy=0.0


100%|██████████| 1414/1414 [00:45<00:00, 31.36it/s]


[('7ZNb7', '7YN7'), ('OULk3', 'CUHs'), ('PktWE', 'TamWP'), ('Zmxjh', 'Zmwmh'), ('bpzSD', 'B723E'), ('Aeaeu', 'AGdqu'), ('C32cU', 'Cszu'), ('lkIgN', 'TjIdN'), ('a7WR5', 'cvnE3'), ('ErzCl', 'E4zGU')]
Epoch=1, Train Loss=2.975743060704297, Test Loss=2.795687286317601 Accuracy=0.00017688157778367384


100%|██████████| 1414/1414 [00:45<00:00, 31.41it/s]


[('7ZNb7', '7Ny7'), ('OULk3', 'CUHs'), ('PktWE', 'PxmWF'), ('Zmxjh', 'ZmAh'), ('bpzSD', 'BPZSD'), ('Aeaeu', 'ACdGu'), ('C32cU', '6szcu'), ('lkIgN', 'Tlev'), ('a7WR5', 'gvME3'), ('ErzCl', 'ERZGu')]
Epoch=2, Train Loss=2.835119083223853, Test Loss=2.6292138586961564 Accuracy=0.00017688157778367384


 23%|██▎       | 2950/12720 [04:41<15:26, 10.54it/s]

In [ ]:
    state = {
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    }

    torch.save(state, f'{drive_dir}state_10.pth')